## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-31-22-49-30 +0000,startribune,Twins send former All-Star shortstop Carlos Co...,https://www.startribune.com/twins-trade-carlos...
1,2025-07-31-22-41-44 +0000,nyt,Fed Up With Netanyahu and Handling of Gaza War...,https://www.nytimes.com/2025/07/31/us/politics...
2,2025-07-31-22-39-03 +0000,latimes,Fugitive linked to South American bank robbery...,https://www.latimes.com/california/story/2025-...
3,2025-07-31-22-38-36 +0000,nypost,Chaos erupts after daughter of detained Long I...,https://nypost.com/2025/07/31/us-news/chaos-er...
4,2025-07-31-22-37-50 +0000,nypost,Wife of torched Va. Councilman Lee Vogler call...,https://nypost.com/2025/07/31/us-news/virginia...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2330/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
73,trump,71
112,tariffs,25
10,trade,16
190,tariff,14
207,palestinian,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
232,2025-07-31-10-33-13 +0000,cbc,"U.S. extends trade deal with Mexico, sets tari...",https://www.cbc.ca/news/world/trump-tariffs-so...,157
159,2025-07-31-16-29-38 +0000,latimes,New Trump tariffs on Mexico delayed 90 days as...,https://www.latimes.com/world-nation/story/202...,137
110,2025-07-31-19-12-10 +0000,nyt,Appeals Court Hears Legal Challenge to Trump T...,https://www.nytimes.com/2025/07/31/business/tr...,135
178,2025-07-31-15-31-00 +0000,wsj,President Trump’s tighter deadline for Russia ...,https://www.wsj.com/politics/policy/trumps-rus...,133
80,2025-07-31-20-11-17 +0000,wapo,Trump puts trade deal in doubt after Canada ba...,https://www.washingtonpost.com/politics/2025/0...,129


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
232,157,2025-07-31-10-33-13 +0000,cbc,"U.S. extends trade deal with Mexico, sets tari...",https://www.cbc.ca/news/world/trump-tariffs-so...
89,57,2025-07-31-20-03-00 +0000,wsj,President Trump is building a Mar-a-Lago style...,https://www.wsj.com/politics/policy/trump-ball...
258,54,2025-07-31-08-34-03 +0000,nypost,"Russian missile and drone attack batters Kyiv,...",https://nypost.com/2025/07/31/world-news/russi...
68,50,2025-07-31-20-46-40 +0000,nypost,NYC shooting victim Wesley LePatner’s teen dau...,https://nypost.com/2025/07/31/us-news/teen-dau...
292,45,2025-07-31-03-54-00 +0000,wsj,Senators rejected an effort to block U.S. arms...,https://www.wsj.com/politics/policy/republican...
178,43,2025-07-31-15-31-00 +0000,wsj,President Trump’s tighter deadline for Russia ...,https://www.wsj.com/politics/policy/trumps-rus...
198,41,2025-07-31-14-01-19 +0000,nypost,Fed’s preferred inflation gauge ticks up as ta...,https://nypost.com/2025/07/31/business/feds-pr...
231,39,2025-07-31-10-37-06 +0000,bbc,Canada follows France and UK with plan to reco...,https://www.bbc.com/news/articles/ceqyx35d9x2o...
27,34,2025-07-31-22-00-31 +0000,nyt,County Emergency Official Says He Was Ill and ...,https://www.nytimes.com/2025/07/31/us/politics...
69,34,2025-07-31-20-45-00 +0000,wsj,President Trump’s popular policy will cut taxe...,https://www.wsj.com/politics/policy/no-tax-ove...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
